In [23]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import os
import pickle

if (not os.path.exists("depts")):
    os.mkdir("depts")

YEAR = 22
terms = [f'FA{YEAR-1}', f'WI{YEAR}', F'SP{YEAR}', F'S1{YEAR}', f'S2{YEAR}', f'S3{YEAR}']

CAPEURL = 'https://cape.ucsd.edu/responses/Results.aspx'
CAPETITLE = 'Course And Professor Evaluations (CAPE)'

with open('all_depts.pick', 'rb') as f:
    all_depts = pickle.load(f)

# taken from BetterCapes
# https://github.com/andportnoy/smartercapes.com/blob/master/tools.py
def get_raw_cape_dataframe(dept:str):

    options = webdriver.ChromeOptions()
    options.add_argument("user-data-dir=C:/Users/kalki/AppData/Local/Google/Chrome/User Data")
    options.add_argument("profile-directory=Profile 7")
    driver = webdriver.Chrome(options=options)
    
    # driver.get(f'https://cape.ucsd.edu/responses/Results.aspx?Name={course[0]}+{course[1]}&CourseNumber=')
    driver.get(f'https://cape.ucsd.edu/responses/Results.aspx?Name=&CourseNumber={dept}')
    wait = WebDriverWait(driver, 60)
    element = wait.until(expected_conditions.title_contains(CAPETITLE))
    # read in the dataset from the html file
    df = pd.read_html(driver.page_source)[0]
    driver.quit()

    return df

def clean_df(df, dept):
    df = (
        df[
            ['Instructor', 'Course', 'Term', 
            'Avg Grade Received', 'Enroll']
        ]
        .assign(Course = df.get('Course').str.split(' - ').apply(lambda x: x[0]))
    )

    yearly = df[df.Term.isin(terms)].groupby('Course').sum()
    def get_yearly_students(course):
        if course not in yearly.index:
            return 0
        return yearly.get('Enroll').loc[course]

    df = df.dropna()

    df = (df
        .assign(
            GPA=(df.get('Avg Grade Received')
                .str.split('(')
                .apply(lambda x : x[-1])
                .str.rstrip(")")
                .astype('float'))
        )
    )
    df = df.assign(total_grade_points = df.get('GPA')*df.get('Enroll'))
    df = df.groupby('Course').sum().reset_index()
    df = (df
        .assign(yearly_num=df.get('Course').apply(get_yearly_students))
        .assign(GPA = df.get('total_grade_points')/df.get('Enroll'))
        .drop(columns=['total_grade_points'])
        .assign(dept=df.get('Course').str.split(' ').apply(lambda x: x[0]))
        .assign(num=df.get('Course').str.split(' ').apply(lambda x: x[1]))
    )

    df = df[df.get('dept') == dept]
    df = df[df.get('yearly_num')>0]

    def remove_str(s:str):
        if s[-1].isalpha():
            return remove_str(s[:-1])
        return s
    df = df.assign(num = df.get('num').apply(remove_str).astype("int"))
    df = df.sort_values(by=['num', 'Course']).set_index('Course').reset_index()
    df = df.assign(index=df.index).set_index('Course')
    return df

In [24]:

dfs = []
for dept in all_depts:
    if os.path.exists(f'depts/{dept}.csv'):
        clean = pd.read_csv(f'depts/{dept}.csv')
    else:
        raw = get_raw_cape_dataframe(dept)
        clean = clean_df(raw, dept)
        clean.to_csv(f'depts/{dept}.csv')
        
    dfs.append(clean)

data = pd.concat(dfs)
data.to_csv('data.csv')

C:\Users\kalki\AppData\Local\Temp\ipykernel_7224\1726617474.py:48: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  yearly = df[df.Term.isin(terms)].groupby('Course').sum()
C:\Users\kalki\AppData\Local\Temp\ipykernel_7224\1726617474.py:66: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('Course').sum().reset_index()
C:\Users\kalki\AppData\Local\Temp\ipykernel_7224\1726617474.py:48: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid fo

In [25]:
data

,Enroll,GPA,yearly_num,dept,num,index
Course,,,,,,
AAS 10,519,3.651734,199,AAS,10,0
AAS 190,83,4.000000,83,AAS,190,1
ANAR 116,24,3.350000,16,ANAR,116,0
ANAR 135,22,4.000000,22,ANAR,135,1
ANAR 143,216,3.420694,36,ANAR,143,2
...,...,...,...,...,...,...
VIS 183B,440,3.593932,23,VIS,183,71
VIS 185,65,3.558923,19,VIS,185,72
WCWP 10A,9572,3.229711,1003,WCWP,10,0


In [14]:
data.sort_values(by='GPA')

,Course,GPA,yearly_num,dept,num,index
6,ECE 35,2.474312,647,ECE,35,6
62,SOCI 168G,2.480000,26,SOCI,168,62
36,CHEM 130,2.581061,110,CHEM,130,36
4,SE 101B,2.598556,121,SE,101,4
11,ECE 102,2.605461,150,ECE,102,11
...,...,...,...,...,...,...
66,PSYC 194B,4.000000,30,PSYC,194,66
2,TDPF 163,4.000000,11,TDPF,163,2
9,MAE 94,4.000000,26,MAE,94,9
8,ECON 100CH,4.000000,14,ECON,100,8


In [41]:
temp = data.assign(total_points=data.get('GPA')*data.get('Enroll'))
temp = temp.groupby('dept').sum().dropna()
temp = temp.assign(GPA=temp.get('total_points') / temp.get('Enroll')).drop(columns='total_points')
temp.sort_values('GPA')

,Enroll,GPA,yearly_num,num,index
dept,,,,,
ECON,257284,2.874236,15647,5803,1176
MATH,358972,2.879947,35929,8815,2628
AWP,3346,2.930702,2287,17,3
CHEM,244732,2.941767,23725,4922,1326
ECE,79635,2.943873,7453,7248,1596
...,...,...,...,...,...
LTIT,345,3.905391,36,54,3
CCS,136,3.924191,84,326,3
TDDM,287,3.971220,97,106,3
